# Downloading DPA RegioLine press releases

As a possible better alternative to news articles. For example, they can be expected to be de-duplicated already, and they should have a balanced geographical distribution, and only refer to events within Germany, and clearly state the location.

In [1]:
from protest_impact.data.protests.config import search_string

search_string

'protest* OR demo OR demonstr* OR kundgebung OR versamm* OR "soziale bewegung" OR hausbesetz* OR streik* OR unterschriften* OR petition OR hasskriminalität OR unruhen OR aufruhr OR aufstand OR rebell* OR blockade OR blockier* OR sitzblock* OR boykott* OR riot OR aktivis* OR bürgerinitiative OR bürgerbegehren OR marsch OR aufmarsch OR parade OR mahnwache OR hungerstreik OR "ziviler ungehorsam"'

In [1]:
from calendar import monthrange
from os import listdir
from time import sleep

from pyperclip import paste

download_dir = "/Users/david/Downloads"
n_files = len(listdir(download_dir))
print(n_files)

427


In [36]:
from subprocess import run


def click(args: str):
    run(["cliclick", "-e 1000", *args.split(" ")])


def move(x: int, y: int):
    click(f"m:{x},{y}")

In [ ]:
sleep(1)
click("kd:cmd kp:tab ku:cmd")
sleep(1)

year = 2017
month = 9
i = 2
while year > 2012:
    sleep(5)
    while month > 0:
        sleep(5)
        move(362, 373)
        click("c:.")
        sleep(10)
        move(250, 670)
        click("c:.")
        sleep(1)
        click("kd:cmd t:a ku:cmd")
        sleep(1)
        click(f"t:01/{str(month).zfill(2)}/{year}")
        sleep(1)
        move(250, 700)
        click("c:.")
        sleep(1)
        click("kd:cmd t:a ku:cmd")
        sleep(1)
        last_day = monthrange(year, month)[1]
        click(f"t:{str(last_day).zfill(2)}/{str(month).zfill(2)}/{year}")
        sleep(1)
        move(330, 735)
        click("c:.")
        sleep(7)
        move(490, 203)
        click("dc:.")
        sleep(1)
        click("kd:cmd t:c ku:cmd")
        sleep(1)
        clipboard = paste()
        print("📋", clipboard)
        max_items = int(clipboard)
        while i <= (max_items - 1) // 100:
            sleep(5)
            move(622, 240)
            click("c:.")
            sleep(1)
            move(500, 305)
            click("c:.")
            sleep(1)
            end = min((i + 1) * 100, max_items)
            if i * 100 + 1 == end:
                click(f"ku:cmd t:{str(i*100+1)}")
            else:
                click(f"ku:cmd t:{str(i*100+1)}-{str(end)}")
            sleep(1)
            # move(500, 720)
            # click("dc:.")
            # sleep(1)
            # click(f'ku:cmd t:dpa-regio-2013-2022_{str(year)}-{str(month).zfill(2)}')
            # sleep(1)
            n_files = len(listdir(download_dir))
            move(900, 810)
            click("c:.")  # Download
            sleep(5)
            while len(listdir(download_dir)) == n_files:
                sleep(10)
            print(year, month, i)
            sleep(5)
            i += 1
        i = 0
        month -= 1
    month = 12
    year -= 1

click("kd:cmd kp:tab ku:cmd")

In [3]:
import json
import re
from pathlib import Path
from zipfile import ZipFile

from dateparser import parse
from striprtf.striprtf import rtf_to_text
from tqdm.notebook import tqdm

from protest_impact.util import project_root

items = []
fails = []
for i, file in tqdm(list(enumerate(Path(download_dir).glob("Files*.ZIP")))):
    with ZipFile(file) as zipObj:
        for file in zipObj.filelist:
            rtf = zipObj.read(file).decode("utf-8")
            plaintext = rtf_to_text(rtf, errors="ignore").strip()
            plaintext = plaintext.replace("\xa0", " ")
            try:
                title, rest = plaintext.split("dpa RegioLine", 1)
                date, rest = rest.strip().split("\n", 1)
                date = parse(date.strip())
                meta, rest = rest.split("Body", 1)
                if rest.strip().startswith("Zusammenfassung\n"):
                    _, summary, rest = rest.strip().split("\n", 2)
                else:
                    summary = ""
                location, rest = re.split(r" ?\(dpa| ?\(dap", rest, 1)
                if rest.startswith("/"):
                    region, rest = re.split(r"\) ?- ?|\) ?– ?", rest[1:], 1)
                else:
                    region = ""
                    rest = rest[4:]
                if "Graphic" in rest:
                    text, graphic = rest.split("Graphic", 1)
                else:
                    text, rest = rest.split("Load-Date", 1)
                items.append(
                    {
                        "title": title.strip(),
                        "date": date.isoformat(),
                        "summary": summary.strip(),
                        "location": location.strip(),
                        "region": region.strip(),
                        "text": text.strip(),
                    }
                )
            except Exception as e:
                fails.append(plaintext)
    if i % 30 == 0:
        with open(project_root / f"data/news/dpa_{i}.jsonl", "w") as f:
            for item in items:
                f.write(json.dumps(item) + "\n")

with open(project_root / "data/news/dpa.jsonl", "w") as f:
    for item in items:
        f.write(json.dumps(item) + "\n")

len(items), len(fails)

  0%|          | 0/379 [00:00<?, ?it/s]

(33060, 557)

## Further available articles

Search for "Klima*" from 2020-01 yields the following counts:

Rheinische Post
29,214

Süddeutsche Zeitung (inkl. Regionalausgaben)
27,044

Nordwest-Zeitung
15,344

Frankfurter Rundschau
14,307

Kölner Stadt-Anzeiger
14,134

Kölnische Rundschau
13,721

Neue Zürcher Zeitung (Internationale Ausgabe) & NZZ am Sonntag
12,938

Schwarzwälder Bote
12,835

Stuttgarter Zeitung
11,453

Der Tagesspiegel
11,134

taz, die tageszeitung
10,090

General-Anzeiger (Bonn)
9,425

Stuttgarter Nachrichten
9,212

Aachener Zeitung
8,704

Aachener Nachrichten
8,595

Allgemeine Zeitung (Germany)
8,571

Die Presse
8,394

Sächsische Zeitung Regionalausgaben
8,024

Frankenpost
7,235

Neuss Grevenbroicher Zeitung
7,098

Kronen Zeitung
6,845

Bergische Morgenpost
6,722

Darmstädter Echo
6,387

Wiesbadener Kurier (Germany)
6,248

Der Standard
6,104

Südwest Presse
6,063

Groß-Gerauer Echo
6,000

Die Welt
5,729

Starkenburger Echo
5,722

Bürstädter Zeitung (Germany)
5,721

Lampertheimer Zeitung (Germany)
5,721

Odenwälder Echo
5,679

Main-Spitze (Germany)
5,619

NON Niederosterr. Nachrichten
5,177

Berliner Zeitung
5,061

Märkische Oderzeitung Frankfurter Stadtbote (Oder)
4,941

Oranienburger Generalanzeiger
4,917

Kurier
4,843

Wormser Zeitung (Germany)
4,793

Idsteiner Zeitung (Germany)
4,787

Die ZEIT (inklusive ZEIT Magazin)
4,706

Lausitzer Rundschau
4,624

Passauer Neue Presse (Stadt und Landkreis Passau)
4,261

Handelsblatt
4,211

Luzerner Zeitung
4,100

Tagblatt für den Kanton Thurgau
3,949

Tages-Anzeiger
3,918

Frankfurter Neue Presse (Regionalausgaben)
3,906

St.Galler Tagblatt (Stammausgabe)
3,874

Obwaldner Zeitung
3,869

Nidwaldner Zeitung
3,864

Zuger Zeitung
3,851

Urner Zeitung
3,814

Oberhessische Zeitung (Germany)
3,809

Wiler Zeitung
3,797

Kleine Zeitung
3,743

BILD Regionalausgaben
3,742

Appenzeller Zeitung
3,726

OO Nachrichten
3,716

Gießener Anzeiger
3,696

Luxemburger Wort
3,692

Lauterbacher Anzeiger (Germany)
3,648

Toggenburger Tagblatt
3,642

Rheintaler
3,559

Mitteldeutsche Zeitung
3,444

Energie & Management
3,428

Die Welt am Sonntag
3,308

Börsen-Zeitung
3,263

Frankfurter Neue Presse
3,131

Solinger Morgenpost
3,076

Tiroler Tageszeitung und TT am Sonntag
3,008

Werdenberger Obertoggenburger
3,004

Salzburger Nachrichten
2,969

Der Bund
2,635

Kreis Anzeiger
2,419

Berner Zeitung
2,419

Basler Zeitung
2,318

Wiener Zeitung
2,223

Bote der Urschweiz
2,218

Nürnberger Nachrichten
2,153

Usinger Anzeiger
2,134

Kölner Express
2,074

VN Vorarlberger Nachrichten
2,061

Hamburger Morgenpost
2,053

B. Z.
2,048

Langenthaler Tagblatt
2,029

Der Landbote
2,024

Thuner Tagblatt
1,970

Dolomiten
1,961

Berner Oberlander
1,933

Zurichsee-Zeitung
1,852

Berliner Kurier
1,846

Nürnberger Zeitung
1,817

Bayerische Staatszeitung
1,639

Zurcher Unterlander
1,639

Oberösterreichisches Volksblatt
1,472

Alt-Neuöttinger Anzeiger
1,434

Wetterauer Zeitung
1,375

Lebensmittel Zeitung
1,343

Täglicher Hafenbericht
1,342

Deggendorfer Zeitung
1,240

Blick
1,224

SonntagsZeitung
1,140

NEUE Vorarlberger Tageszeitung
1,135

Passauer Neue Presse (Freyung-Grafenau)
1,112

Der Bayerwald-Bote
1,053